In [6]:
import openai

In [7]:
question = "最近涉及大额资金的事件有哪些？"

# 计算输入问题的 embedding

In [8]:
openai.api_key = "sk-xxxxx"

resp = openai.Embedding.create(
    model="text-embedding-ada-002",
    input=question,
)
embedding = resp["data"][0]["embedding"]

len(embedding)

1536

# 从数据库中内容比较相似度

In [9]:
import psycopg2
from psycopg2.extras import DictCursor

In [10]:
similarity_threshold = 0.78
max_matched_doc_counts = 3
conn = psycopg2.connect(database="chi", host="localhost", user="chi", password="", port="5432")

similarity_search_sql = f"""
SELECT title, content, token_size, 1 - (embedding <=> '{embedding}') AS similarity 
FROM docs WHERE 1 - (embedding <=> '{embedding}') > {similarity_threshold} ORDER BY similarity DESC LIMIT {max_matched_doc_counts};
"""

cur = conn.cursor(cursor_factory=DictCursor)
cur.execute(similarity_search_sql)
matched_docs = cur.fetchall()

matched_docs

[['9. 美国恶意欺骗者“窃取”新冠救济金，涉案金额或超4000亿美元',
  '美国历史上最大的骗局正在发生，一些恶意欺骗者利用种种理由窃取了美国联邦的新冠救济金，涉案金额可能超过4000亿美金。美联社6月12日发布的一项分析发现，这些诈骗者可能窃取了超过2800亿美元的新冠救援资金；另有1230亿美元被浪费或误用。这些数额加起来，损失超过4000亿美元，占美国政府迄今为止发放的4.2万亿美元新冠救济援助的10%。\nhttps://finance.sina.com.cn/world/2023-06-13/doc-imyxcqei8863710.shtml',
  211,
  0.8087266155636635],
 ['10. 2.9亿美元！摩根大通与“爱泼斯坦案”受害者达成和解',
  '据《华尔街日报》中文网13日报道，摩根大通将支付2.9亿美元和解爱泼斯坦性侵受害者提起的诉讼。律师们表示，这项和解协议将成为有史以来性交易民事案件最大金额和解之一。摩根大通高管被问及该行多年来与爱泼斯坦的交易情况后不久，爱泼斯坦原告的律师表示，该行同意支付2.9亿美元，以和解与爱泼斯坦关系有关的诉讼。代表指控爱泼斯坦性侵的女性提起的诉讼，帮助揭露了该银行在爱泼斯坦被定罪多年后与其关系细节，迫使首席执行官杰米·戴蒙在宣誓后回答问题，并导致该银行转而起诉了前摩根大通首席执行官Jes Staley。\nhttp://www.jwview.com/jingwei/html/06-13/544666.shtml',
  325,
  0.8060536909896684],
 ['4. 第四度被罚 牛散任良成被罚没近3亿',
  '日前，中国证监会公布的一则行政处罚决定书显示，有着初代股民之称的“牛散”任良成被罚没近3亿元。据处罚决定书显示，2015年4月至2015年12月，任良成先后控制使用陈某华、陈某（甲）等31人名下的51个账户，操纵酒钢宏兴、贵人鸟、美锦能源、太化股份等12只股票的交易价格和交易量。2016年8月至2017年10月，任良成先后控制使用蔡某农、陈某华等98人名下的150个账户操纵远兴能源、隆华节能、冠农股份、维维股份等4只股票交易价格和交易量。经统计，其仅操纵“酒钢宏兴”一只个股盈利就逾2亿元。根据前述复核意见，剔除涉及刑事案件的“酒钢宏兴”后，任良成操纵“

# 交给 OpenAI 整合结果

In [11]:
docs = "\n\n".join(i["title"] + "\n" + i["content"] for i in matched_docs)

prompt = f"""\
{docs}

请基于以上内容回答问题：{question}
请直接回答结果，不需要推理。
如果你并不知道答案，请不要编造答案。
"""

print(prompt)

9. 美国恶意欺骗者“窃取”新冠救济金，涉案金额或超4000亿美元
美国历史上最大的骗局正在发生，一些恶意欺骗者利用种种理由窃取了美国联邦的新冠救济金，涉案金额可能超过4000亿美金。美联社6月12日发布的一项分析发现，这些诈骗者可能窃取了超过2800亿美元的新冠救援资金；另有1230亿美元被浪费或误用。这些数额加起来，损失超过4000亿美元，占美国政府迄今为止发放的4.2万亿美元新冠救济援助的10%。
https://finance.sina.com.cn/world/2023-06-13/doc-imyxcqei8863710.shtml

10. 2.9亿美元！摩根大通与“爱泼斯坦案”受害者达成和解
据《华尔街日报》中文网13日报道，摩根大通将支付2.9亿美元和解爱泼斯坦性侵受害者提起的诉讼。律师们表示，这项和解协议将成为有史以来性交易民事案件最大金额和解之一。摩根大通高管被问及该行多年来与爱泼斯坦的交易情况后不久，爱泼斯坦原告的律师表示，该行同意支付2.9亿美元，以和解与爱泼斯坦关系有关的诉讼。代表指控爱泼斯坦性侵的女性提起的诉讼，帮助揭露了该银行在爱泼斯坦被定罪多年后与其关系细节，迫使首席执行官杰米·戴蒙在宣誓后回答问题，并导致该银行转而起诉了前摩根大通首席执行官Jes Staley。
http://www.jwview.com/jingwei/html/06-13/544666.shtml

4. 第四度被罚 牛散任良成被罚没近3亿
日前，中国证监会公布的一则行政处罚决定书显示，有着初代股民之称的“牛散”任良成被罚没近3亿元。据处罚决定书显示，2015年4月至2015年12月，任良成先后控制使用陈某华、陈某（甲）等31人名下的51个账户，操纵酒钢宏兴、贵人鸟、美锦能源、太化股份等12只股票的交易价格和交易量。2016年8月至2017年10月，任良成先后控制使用蔡某农、陈某华等98人名下的150个账户操纵远兴能源、隆华节能、冠农股份、维维股份等4只股票交易价格和交易量。经统计，其仅操纵“酒钢宏兴”一只个股盈利就逾2亿元。根据前述复核意见，剔除涉及刑事案件的“酒钢宏兴”后，任良成操纵“漳州发展”等7只股票合计盈利7413.94万元。依据2005年《证券法》第203条规定，中国证监会决定，没收任良成违法所得7413.94万元，并处以2.22亿元罚款。
今年已有

In [12]:
openai.api_key = "sk-xxxxx"

resp = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)

print(resp["choices"][0]["message"]["content"])

1. 美国恶意欺骗者“窃取”新冠救济金，涉案金额或超4000亿美元。
2. 摩根大通将支付2.9亿美元和解爱泼斯坦性侵受害者提起的诉讼。
3. 牛散任良成被罚没近3亿元。
